In [1]:
import requests
import pandas as pd

headers = {
    'Content-Type': 'application/json'
}
requestResponse = requests.get("https://api.tiingo.com/iex/aapl/prices?startDate=2019-01-02&resampleFreq=5min&columns=open,high,low,close,volume&token=1bb8adbd31a937981ecc6d299b941b788e0ef3d8", headers=headers)
print(requestResponse.json())

[{'date': '2025-05-30T18:40:00.000Z', 'open': 199.14, 'high': 199.155, 'low': 198.845, 'close': 199.015, 'volume': 7981.0}, {'date': '2025-05-30T18:45:00.000Z', 'open': 199.02, 'high': 199.225, 'low': 198.995, 'close': 199.1, 'volume': 6453.0}, {'date': '2025-05-30T18:50:00.000Z', 'open': 199.105, 'high': 199.33, 'low': 199.06, 'close': 199.22, 'volume': 5274.0}, {'date': '2025-05-30T18:55:00.000Z', 'open': 199.22, 'high': 199.38, 'low': 199.12, 'close': 199.335, 'volume': 6846.0}, {'date': '2025-05-30T19:00:00.000Z', 'open': 199.31, 'high': 199.6, 'low': 199.28, 'close': 199.54, 'volume': 9109.0}, {'date': '2025-05-30T19:05:00.000Z', 'open': 199.54, 'high': 199.825, 'low': 199.53, 'close': 199.635, 'volume': 10234.0}, {'date': '2025-05-30T19:10:00.000Z', 'open': 199.64, 'high': 199.83, 'low': 199.37, 'close': 199.64, 'volume': 7740.0}, {'date': '2025-05-30T19:15:00.000Z', 'open': 199.635, 'high': 200.095, 'low': 199.615, 'close': 199.915, 'volume': 11165.0}, {'date': '2025-05-30T19:20

In [2]:
pd.DataFrame(requestResponse.json()).tail()

,date,open,high,low,close,volume
9995,2025-11-26T20:35:00.000Z,278.030,278.055,277.815,277.820,22668.0
9996,2025-11-26T20:40:00.000Z,277.820,278.300,277.790,278.175,14452.0
9997,2025-11-26T20:45:00.000Z,278.175,278.280,277.945,278.070,18257.0
9998,2025-11-26T20:50:00.000Z,278.040,278.045,277.195,277.255,20869.0
9999,2025-11-26T20:55:00.000Z,277.360,277.920,277.325,277.570,57088.0


In [ ]:
headers = {
    'Content-Type': 'application/json'
}
requestResponse = requests.get("https://api.tiingo.com/tiingo/news?token=1bb8adbd31a937981ecc6d299b941b788e0ef3d8", headers=headers)
print(requestResponse.json())

In [ ]:
news_df = pd.DataFrame(requestResponse.json())

In [ ]:
news_df

In [1]:
import requests
import pandas as pd
import time

# 1. Setup: The SEC requires a User-Agent string with your email (for administrative contact)
headers = {'User-Agent': 'JustinNewberry (jstnnwbrr@gmail.com)'}

def get_sec_data():
    print("Fetching Master Ticker List from SEC...")
    # Get the list of all tickers (CIK, Ticker, Title)
    tickers_url = "https://www.sec.gov/files/company_tickers.json"
    response = requests.get(tickers_url, headers=headers)
    
    if response.status_code != 200:
        print(f"Error fetching list: {response.status_code}")
        return
    
    # Parse the JSON into a DataFrame
    data = response.json()
    company_list = [value for key, value in data.items()]
    df = pd.DataFrame(company_list)
        
    # 2. Create a DataFrame
    df = pd.DataFrame(company_list)
    df.rename(columns={'cik_str': 'CIK', 'ticker': 'Ticker', 'title': 'Company'}, inplace=True)
    
    print(f"Found {len(df)} companies. Fetching Industry data...")
    
    # 3. Fetch Industry (SIC Code) for each company
    # It will take about 15-20 minutes to run for all companies due to rate limits.
    
    industries = []
    sic_codes = []
    
    for cik in df['CIK']: 
        # Pad CIK with zeros to 10 digits
        padded_cik = str(cik).zfill(10)
        url = f"https://data.sec.gov/submissions/CIK{padded_cik}.json"
        
        try:
            r = requests.get(url, headers=headers)
            if r.status_code == 200:
                company_data = r.json()
                industries.append(company_data.get('sicDescription', 'N/A'))
                sic_codes.append(company_data.get('sic', 'N/A'))
            else:
                industries.append("Error")
                sic_codes.append("Error")
        except Exception as e:
            industries.append("Error")
            sic_codes.append("Error")
            
        # Sleep to respect SEC rate limit (max 10 requests/sec)
        time.sleep(0.11) 

    # Add columns to DataFrame
    df_final = df.copy()
    df_final['SIC Code'] = sic_codes
    df_final['Industry (SEC)'] = industries
    
    # Display the result
    print(df_final)
    
    # Export to CSV
    df_final.to_csv("C:\\Users\\sar81\\Desktop\\sec_companies_list.csv", index=False)
    print("Saved to sec_companies_list.csv")

get_sec_data()

Fetching Master Ticker List from SEC...
Found 10182 companies. Fetching Industry data...
           CIK Ticker                     Company SIC Code  \
0      1045810   NVDA                 NVIDIA CORP     3674   
1       320193   AAPL                  Apple Inc.     3571   
2       789019   MSFT              MICROSOFT CORP     7372   
3      1652044  GOOGL               Alphabet Inc.     7370   
4      1018724   AMZN              AMAZON COM INC     5961   
...        ...    ...                         ...      ...   
10177  1980845  ENGNW        enGene Holdings Inc.     2836   
10178  1718224  BTBDW             BT Brands, Inc.     5812   
10179  1341317  BWBBP  Bridgewater Bancshares Inc     6022   
10180  1918661  ESHAR       ESH Acquisition Corp.     6770   
10181  1842566  AISPW   Airship AI Holdings, Inc.     7372   

                                          Industry (SEC)  
0                       Semiconductors & Related Devices  
1                                   Electronic C